# Finetune LLM using HuggingFace PEFT library, transformers and bitsandbytes.

Notebook Goal: How to fine-tune LLM using peft library and bitsandbytes for loading Larger models in 8 bit. (using LoRA)

## Installing Dependencies

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

## Loading Model

1. load **opt-6.7b** model(float16) with 13GB weigth on the Hub.
2. loading it in 8 bit (to save around 6 GB)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b",
                                             load_in_8bit= True,
                                             device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

# Post-processing on the 8-bit model.

1. freeze all our layers
2. cast the layer-norm in float32 for stability

In [ ]:
for param in model.parameters():
  # freeze
  param.requires_grad = False
  if param.ndim == 1:
    #cast small parameters to fp32 for stability
    param.data = param.data.to(torch.float32)

# reduce number of stored activations ( a memory-saving technique)
model.gradient_checkpointing_enable()
# ensures gradient are calculated for model's input tensors.
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self,x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

# LoRA

In [ ]:
# using peft library and applying LoRA using get_peft_model utility function.
def print_trainable_parameters(model):
  trainable_parameters = 0
  all_param = 0
  for _,param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  print(
      f"trainable parameters:{trainable_parameters} || all params: {all_param} || trainable%: {100*trainable_parameters/all_param}"
  )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'CASUAL_LM'
)

In [ ]:
model = get_peft_model(model, config)

In [ ]:
print_trainable_parameters(model)

trainable parameters:8388608 || all params: 6666862592 || trainable%: 0.12582542214183376


# Training

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [ ]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = 'outputs'
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.250000
2,2.386600
3,2.195800
4,2.228900
5,1.766200
6,2.358300
7,2.338200
8,2.198600
9,2.175700
10,2.403200


TrainOutput(global_step=200, training_loss=1.9264344310760497, metrics={'train_runtime': 2301.3217, 'train_samples_per_second': 1.391, 'train_steps_per_second': 0.087, 'total_flos': 1.164184860229632e+16, 'train_loss': 1.9264344310760497, 'epoch': 1.28})

# Sharing on the HuggingFace Hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("SumitxThokar/opt-6.7b-lora-version1", use_auth_token= True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SumitxThokar/opt-6.7b-lora-version1/commit/75a1f6d96d448a0c8587e500de3f607d0aa08bcd', commit_message='Upload model', commit_description='', oid='75a1f6d96d448a0c8587e500de3f607d0aa08bcd', pr_url=None, pr_revision=None, pr_num=None)